# Pronósticos Mundial - Graphs

**Date**: 1122

**Data source**: https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017?resource=download

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [2]:
encode = 'ISO-8859-16'

In [3]:
print(os.getcwd())

/Users/jpolanco/Desktop/aws


In [4]:
path = "/Users/jpolanco/Downloads/archive/"

In [5]:
files = os.listdir(path)
files = pd.DataFrame({"file":files})
files = files.loc[files['file'] != '.DS_Store']

files.sort_values(by='file').tail()

,file
1,results.csv
0,shootouts.csv


In [6]:
db = pd.read_csv(path+"results.csv")
db.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [7]:
db["date"] = pd.to_datetime(db["date"], format="%Y-%m-%d").dt.year

In [8]:
#db_2 = pd.read_csv(path+"shootouts.csv")
#db_2.head()

In [9]:
df = db[(db["tournament"] == "FIFA World Cup")&(db["date"] >= 1998)]
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
21634,1998,Brazil,Scotland,2.0,1.0,FIFA World Cup,Saint-Denis,France,True
21635,1998,Morocco,Norway,2.0,2.0,FIFA World Cup,Montpellier,France,True
21636,1998,Cameroon,Austria,1.0,1.0,FIFA World Cup,Toulouse,France,True
21637,1998,Italy,Chile,2.0,2.0,FIFA World Cup,Bordeaux,France,True
21638,1998,France,South Africa,3.0,0.0,FIFA World Cup,Marseille,France,False


In [10]:
# Históricos
unique_countries = pd.DataFrame(list(df.home_team.unique())+list(df.away_team.unique())).drop_duplicates()[0].tolist()
unique_countries[:10]

['Brazil',
 'Morocco',
 'Cameroon',
 'Italy',
 'France',
 'Paraguay',
 'Saudi Arabia',
 'South Korea',
 'Netherlands',
 'Spain']

In [11]:
# Qatar 2022
qatar = ["Qatar", "Ecuador", "Senegal", "Netherlands",
         "England", "Iran", "United States", "Wales",
         "Argentina", "Saudi Arabia", "Mexico", "Poland",
         "France", "Australia", "Denmark", "Tunisia",
         "Spain", "Costa Rica", "Germany", "Japan",
         "Belgium", "Canada", "Morocco", "Croatia",
         "Brazil", "Serbia", "Switzerland", "Cameroon",
         "Portugal", "Ghana", "Uruguay", "South Korea"
        ]

In [12]:
list(set(qatar).difference(unique_countries))

['Wales', 'Canada', 'Qatar']

In [13]:
df['dif_goals'] = df['home_score']-df['away_score']

/var/folders/dc/glgt2cgj2r37zhhr4wjp9_880000gq/T/ipykernel_96602/1392304595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dif_goals'] = df['home_score']-df['away_score']


In [14]:
# Distribution of goal difference
(df['dif_goals'].abs().value_counts()/df['dif_goals'].abs().value_counts().sum())*100

1.0    42.968750
0.0    23.958333
2.0    18.489583
3.0     9.635417
4.0     2.604167
5.0     1.302083
6.0     0.520833
8.0     0.260417
7.0     0.260417
Name: dif_goals, dtype: float64

## Social Network Analysis

In [15]:
df_2 = df[['home_team','away_team','dif_goals']]
df_2.head()

,home_team,away_team,dif_goals
21634,Brazil,Scotland,1.0
21635,Morocco,Norway,0.0
21636,Cameroon,Austria,0.0
21637,Italy,Chile,0.0
21638,France,South Africa,3.0


In [16]:
import networkx as nx

G = nx.from_pandas_edgelist(df_2,
                           source='home_team',
                           target='away_team',
                           edge_attr='dif_goals')

print("No. of unique countries:", len(G.nodes))
print("No. of connections:", len(G.edges))

No. of unique countries: 64
No. of connections: 322


In [17]:
# import pyvis
from pyvis.network import Network
# create vis network
net = Network(notebook=True, width=1000, height=600)
# load the networkx graph
net.from_nx(G)
net.repulsion()
# show
net.show("example.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


**Degree of centrality**:

In [18]:
deg_cen = {}
graphs=[G]

for g in graphs:
    deg_cen[g] = nx.degree_centrality(g)

g_1 = pd.DataFrame(deg_cen)
g_1 = g_1.reset_index()
g_1.columns = ['node','degree_centrality']

In [19]:
g_1 = g_1[g_1['node'].isin(qatar)].sort_values(by="degree_centrality",ascending=False)
g_1

,node,degree_centrality
7,Brazil,0.396825
21,Germany,0.396825
20,France,0.365079
19,England,0.349206
54,Spain,0.333333
53,South Korea,0.317460
2,Argentina,0.301587
33,Netherlands,0.285714
31,Mexico,0.285714
30,Japan,0.285714


**Betweenness Centrality**:

In [20]:
# Número de veces que un nodo actúa como puente a lo largo del camino más corto entre otros dos nodos
betw_cen = {}

for g in graphs:
    betw_cen[g] = nx.betweenness_centrality(g)

g_2 = pd.DataFrame(betw_cen)
g_2 = g_2.reset_index()
g_2.columns = ['node','betweenness_centrality']

In [21]:
g_2 = g_2[g_2['node'].isin(qatar)].sort_values(by="betweenness_centrality",ascending=False)
g_2

,node,betweenness_centrality
7,Brazil,0.108152
19,England,0.080135
54,Spain,0.071574
20,France,0.064188
21,Germany,0.056851
2,Argentina,0.045942
14,Croatia,0.044701
53,South Korea,0.044522
30,Japan,0.041715
42,Portugal,0.038013


**Eigen Vector Centrality**:

In [22]:
# Responde qué tan importante es un nodo dentro de la red social
eig_cen = {}
for g in graphs:
    eig_cen[g] = nx.eigenvector_centrality(g)

g_3 = pd.DataFrame(eig_cen)
g_3 = g_3.reset_index()
g_3.columns = ['node','eigenvector_centrality']

In [23]:
g_3 = g_3[g_3['node'].isin(qatar)].sort_values(by="eigenvector_centrality",ascending=False)
g_3

,node,eigenvector_centrality
21,Germany,0.276493
20,France,0.246697
7,Brazil,0.234769
31,Mexico,0.216105
53,South Korea,0.213599
19,England,0.211290
14,Croatia,0.202866
33,Netherlands,0.202742
2,Argentina,0.195887
42,Portugal,0.187860


## K-Means+SNA

In [24]:
# Pre-processing
G_p1=graphs[0]

# Eliminando nodos que no participarán en 2022
G_p1.remove_nodes_from(list(set(unique_countries).difference(qatar)))

A=nx.adjacency_matrix(G_p1)
L = nx.laplacian_matrix(G_p1).astype(float)

/var/folders/dc/glgt2cgj2r37zhhr4wjp9_880000gq/T/ipykernel_96602/1819177132.py:7: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A=nx.adjacency_matrix(G_p1)
<class 'networkx.utils.decorators.argmap'> compilation 16:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [25]:
G_p1.nodes()

NodeView(('Brazil', 'Morocco', 'Cameroon', 'France', 'Saudi Arabia', 'Denmark', 'South Korea', 'Mexico', 'Netherlands', 'Belgium', 'Spain', 'Argentina', 'Japan', 'Croatia', 'Serbia', 'Iran', 'England', 'Tunisia', 'Germany', 'United States', 'Senegal', 'Uruguay', 'Ecuador', 'Costa Rica', 'Poland', 'Portugal', 'Australia', 'Ghana', 'Switzerland'))

In [26]:
import scipy as sp
w,v = sp.sparse.linalg.eigsh(L, k = 10, which='SM')

In [27]:
from sklearn.cluster import KMeans
X = v*w

kmeans = KMeans(init='k-means++', n_clusters=10, n_init=10)
kmeans.fit_predict(X)

centroids = kmeans.cluster_centers_
labels = kmeans.labels_
error = kmeans.inertia_

#fig = plt.subplots(1, figsize=(12,10))
#colors = ['#d7191c', '#ffffbf', '#2b83ba']
#node_colors = [colors[labels[v]] for v in G_p1.nodes()]
#nx.draw(G_p1, node_color=node_colors, with_labels='True')

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [28]:
k_ex = pd.DataFrame()

k_ex['node'] = pd.DataFrame(G_p1.nodes())
k_ex['group'] = pd.DataFrame(kmeans.fit_predict(X))

k_ex.sort_values(by='group')

,node,group
15,Iran,0
0,Brazil,1
26,Australia,1
18,Germany,1
16,England,1
13,Croatia,1
12,Japan,1
8,Netherlands,1
14,Serbia,1
2,Cameroon,1


**Centralidad vs Grupos**:

In [29]:
resumen = pd.merge(g_3,k_ex,on='node',how='left').sort_values(by=['group','eigenvector_centrality'], 
                                                    ascending=[True,False])

resumen

,node,eigenvector_centrality,group
22,Iran,0.112368,0
0,Germany,0.276493,1
2,Brazil,0.234769,1
5,England,0.211290,1
6,Croatia,0.202866,1
7,Netherlands,0.202742,1
12,Japan,0.160639,1
13,Australia,0.157420,1
14,Denmark,0.155295,1
18,Cameroon,0.136530,1


## Fase de Grupos

In [30]:
a = ["Qatar", "Ecuador", "Senegal", "Netherlands"]
b = ["England", "Iran", "United States", "Wales"]
c = ["Argentina", "Saudi Arabia", "Mexico", "Poland"]
d = ["France", "Australia", "Denmark", "Tunisia"]
e = ["Spain", "Costa Rica", "Germany", "Japan"]
f = ["Belgium", "Canada", "Morocco", "Croatia"]
g = ["Brazil", "Serbia", "Switzerland", "Cameroon"]
h = ["Portugal", "Ghana", "Uruguay", "South Korea"]

**A**:

In [31]:
a_g = resumen[resumen['node'].isin(a)].sort_values(by="eigenvector_centrality", ascending=False)
a_g

,node,eigenvector_centrality,group
7,Netherlands,0.202742,1
21,Ecuador,0.125260,4
27,Senegal,0.078703,7


**B**:

In [32]:
b_g = resumen[resumen['node'].isin(b)].sort_values(by="eigenvector_centrality", ascending=False)
b_g

,node,eigenvector_centrality,group
5,England,0.211290,1
16,United States,0.146597,5
22,Iran,0.112368,0


**C**:

In [33]:
c_g = resumen[resumen['node'].isin(c)].sort_values(by="eigenvector_centrality", ascending=False)
c_g

,node,eigenvector_centrality,group
3,Mexico,0.216105,5
8,Argentina,0.195887,5
23,Saudi Arabia,0.111628,2
25,Poland,0.100071,3


**D**:

In [34]:
d_g = resumen[resumen['node'].isin(d)].sort_values(by="eigenvector_centrality", ascending=False)
d_g

,node,eigenvector_centrality,group
1,France,0.246697,5
13,Australia,0.157420,1
14,Denmark,0.155295,1
26,Tunisia,0.079399,6


**E**:

In [35]:
e_g = resumen[resumen['node'].isin(e)].sort_values(by="eigenvector_centrality", ascending=False)
e_g

,node,eigenvector_centrality,group
0,Germany,0.276493,1
11,Spain,0.181116,5
12,Japan,0.160639,1
17,Costa Rica,0.138126,4


**F**:

In [36]:
f_g = resumen[resumen['node'].isin(f)].sort_values(by="eigenvector_centrality", ascending=False)
f_g

,node,eigenvector_centrality,group
6,Croatia,0.202866,1
15,Belgium,0.149746,5
28,Morocco,0.054777,1


**G**:

In [37]:
g_g = resumen[resumen['node'].isin(g)].sort_values(by="eigenvector_centrality", ascending=False)
g_g

,node,eigenvector_centrality,group
2,Brazil,0.234769,1
18,Cameroon,0.136530,1
19,Switzerland,0.128303,8
20,Serbia,0.126793,1


**H**:

In [38]:
h_g = resumen[resumen['node'].isin(h)].sort_values(by="eigenvector_centrality", ascending=False)
h_g

,node,eigenvector_centrality,group
4,South Korea,0.213599,5
9,Portugal,0.187860,5
10,Uruguay,0.187418,5
24,Ghana,0.109340,9


## 16avos

In [39]:
if a_g.iloc[0,]['eigenvector_centrality']>b_g.iloc[1,]['eigenvector_centrality']:
    m_49 = a_g.iloc[0,]
    print("The winner is "+a_g.iloc[0,]['node'])
else:
    m_49 = b_g.iloc[1,]
    print("The winner is "+b_g.iloc[1,]['node'])
    

The winner is Netherlands


In [40]:
if c_g.iloc[0,]['eigenvector_centrality']>d_g.iloc[1,]['eigenvector_centrality']:
    m_50 = c_g.iloc[0,]
    print("The winner is "+c_g.iloc[0,]['node'])
else:
    m_50 = d_g.iloc[1,]
    print("The winner is "+d_g.iloc[1,]['node'])
    

The winner is Mexico


In [41]:
if e_g.iloc[0,]['eigenvector_centrality']>f_g.iloc[1,]['eigenvector_centrality']:
    m_53 = e_g.iloc[0,]
    print("The winner is "+e_g.iloc[0,]['node'])
else:
    m_53 = f_g.iloc[1,]
    print("The winner is "+f_g.iloc[1,]['node'])
    

The winner is Germany


In [42]:
if g_g.iloc[0,]['eigenvector_centrality']>h_g.iloc[1,]['eigenvector_centrality']:
    m_54 = g_g.iloc[0,]
    print("The winner is "+g_g.iloc[0,]['node'])
else:
    m_54 = h_g.iloc[1,]
    print("The winner is "+h_g.iloc[1,]['node'])
    

The winner is Brazil


In [43]:
if b_g.iloc[0,]['eigenvector_centrality']>a_g.iloc[1,]['eigenvector_centrality']:
    m_51 = b_g.iloc[0,]
    print("The winner is "+b_g.iloc[0,]['node'])
else:
    m_51 = a_g.iloc[1,]
    print("The winner is "+a_g.iloc[1,]['node'])
    

The winner is England


In [44]:
if d_g.iloc[0,]['eigenvector_centrality']>c_g.iloc[1,]['eigenvector_centrality']:
    m_52 = d_g.iloc[0,]
    print("The winner is "+d_g.iloc[0,]['node'])
else:
    m_52 = c_g.iloc[1,]
    print("The winner is "+c_g.iloc[1,]['node'])
    

The winner is France


In [45]:
if f_g.iloc[0,]['eigenvector_centrality']>e_g.iloc[1,]['eigenvector_centrality']:
    m_55 = f_g.iloc[0,]
    print("The winner is "+f_g.iloc[0,]['node'])
else:
    m_55 = e_g.iloc[1,]
    print("The winner is "+e_g.iloc[1,]['node'])
    

The winner is Croatia


In [46]:
if h_g.iloc[0,]['eigenvector_centrality']>g_g.iloc[1,]['eigenvector_centrality']:
    m_56 = h_g.iloc[0,]
    print("The winner is "+h_g.iloc[0,]['node'])
else:
    m_56 = g_g.iloc[1,]
    print("The winner is "+g_g.iloc[1,]['node'])
    

The winner is South Korea


## Cuartos de Final

In [47]:
if m_49['eigenvector_centrality']>m_50['eigenvector_centrality']:
    m_57 = m_49
    print("The winner is "+m_49['node'])
else:
    m_57 = m_50
    print("The winner is "+m_50['node'])
    

The winner is Mexico


In [48]:
if m_53['eigenvector_centrality']>m_54['eigenvector_centrality']:
    m_58 = m_53
    print("The winner is "+m_53['node'])
else:
    m_58 = m_54
    print("The winner is "+m_54['node'])
    

The winner is Germany


In [49]:
if m_51['eigenvector_centrality']>m_52['eigenvector_centrality']:
    m_59 = m_51
    print("The winner is "+m_51['node'])
else:
    m_59 = m_52
    print("The winner is "+m_52['node'])
    

The winner is France


In [50]:
if m_55['eigenvector_centrality']>m_56['eigenvector_centrality']:
    m_60 = m_55
    print("The winner is "+m_55['node'])
else:
    m_60 = m_56
    print("The winner is "+m_56['node'])
    

The winner is South Korea


## Semi-Final

In [51]:
if m_57['eigenvector_centrality']>m_58['eigenvector_centrality']:
    m_61 = m_57
    print("The winner is "+m_57['node'])
else:
    m_61 = m_58
    print("The winner is "+m_58['node'])
    

The winner is Germany


In [52]:
if m_59['eigenvector_centrality']>m_60['eigenvector_centrality']:
    m_62 = m_59
    print("The winner is "+m_59['node'])
else:
    m_62 = m_60
    print("The winner is "+m_60['node'])
    

The winner is France


## Final

In [53]:
if m_61['eigenvector_centrality']>m_62['eigenvector_centrality']:
    print("The winner is "+m_61['node'])
else:
    print("The winner is "+m_62['node'])
    

The winner is Germany
